#  전이학습 기반 분류기

In [1]:
# from PIL import Image
# a = np.array([[1,2,3],[1,2,3],[1,2,3]])
# img = Image.fromarray(a)
# b = np.array(img)

In [1]:
import scipy as sp
import numpy as np
import pandas as pd
from numpy.random import rand
pd.options.display.max_colwidth = 600


from sklearn import preprocessing
from sklearn.metrics import roc_curve, auc, precision_recall_curve
from sklearn.model_selection import train_test_split

import cnn_utils as utils


import matplotlib.pyplot as plt
%matplotlib inline
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 5),
          'axes.labelsize': 'x-large',
          'axes.titlesize':'x-large',
          'xtick.labelsize':'x-large',
          'ytick.labelsize':'x-large'}

plt.rcParams.update(params)


from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tensorflow as tf
from tensorflow.keras import callbacks, optimizers,Model
from tensorflow.keras.applications import vgg16 as vgg
from tensorflow.keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

Using TensorFlow backend.


## 데이터 세트 로딩과 준비

In [3]:
BATCH_SIZE = 64
EPOCHS = 10
NUM_CLASSES = 4
LEARNING_RATE = 1e-4
MOMENTUM = 0.9
PATH="C:\FoodClassification\data"

In [4]:
base_model = vgg.VGG16(weights='imagenet', 
                       include_top=False, 
                       input_shape=(128, 128, 3))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [5]:
# VGG16 모델의 세 번째 블록에서 마지막 층 추출
# VGG16 모델의 네 번째 블록에서 마지막 층 추출
last = base_model.get_layer('block4_pool').output
# last = base_model.get_layer('block3_pool').output

In [6]:
model_save = 'VGG16_128.h5'

## 모델 준비

* 최상위층 없이 VGG16 로딩
* 커스텀 분류기 준비
* 모델의 맨 위에 새로운 층 쌓기

In [7]:
# 상위 층에 분류층 추가
x = GlobalAveragePooling2D()(last)
x= BatchNormalization()(x)
x = Dense(256, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.6)(x)
pred = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(base_model.input, pred)


model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint('model_cnn3-{epoch:03d}-{acc:03f}-{val_acc:03f}.h5', verbose=1,
                                             monitor='val_loss',
                                             save_best_only=True, mode='auto')



In [8]:
for layer in base_model.layers:
     layer.trainable = False

In [9]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=LEARNING_RATE),
              metrics=['accuracy'])

In [10]:
train_datagen = ImageDataGenerator(
                 rescale=1/255,
                validation_split=0.33
                                  )

In [11]:
train_generator = train_datagen.flow_from_directory(
        PATH,
        shuffle=True,
        seed=13,
        target_size=(128,128),
        batch_size=BATCH_SIZE,
        class_mode = 'categorical',
        subset="training")

Found 13350 images belonging to 4 classes.


In [12]:
val_generator = train_datagen.flow_from_directory(
        PATH,
        shuffle=True,
        seed=13,
        target_size=(128,128),
        batch_size=BATCH_SIZE,
        class_mode = 'categorical',
        subset="validation")


Found 6574 images belonging to 4 classes.


In [13]:
train_steps_per_epoch =100
val_steps_per_epoch = 100

history = model.fit_generator(train_generator,
                              steps_per_epoch=train_steps_per_epoch,
                              validation_data=val_generator,
                              validation_steps=val_steps_per_epoch,
                              epochs=EPOCHS,
                              verbose=1)

Epoch 1/10
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
100/100 [==============================] - 453s 5s/step - loss: 0.5748 - acc: 0.7456 - val_loss: 0.4658 - val_acc: 0.7893
Epoch 2/10
100/100 [==============================] - 453s 5s/step - loss: 0.4406 - acc: 0.8040 - val_loss: 0.4142 - val_acc: 0.8143
Epoch 3/10
100/100 [==============================] - 453s 5s/step - loss: 0.3744 - acc: 0.8366 - val_loss: 0.3858 - val_acc: 0.8252
Epoch 4/10
100/100 [==============================] - 452s 5s/step - loss: 0.3455 - acc: 0.8482 - val_loss: 0.3700 - val_acc: 0.8314
Epoch 5/10
100/100 [==============================] - 452s 5s/step - loss: 0.3172 - acc: 0.8671 - val_loss: 0.3605 - val_acc: 0.8371
Epoch 6/10
100/100 [==============================] - 451s 5s/step - loss: 0.3058 - acc: 0.8694 - val_loss: 0.3524 - val_acc: 0.8391
Epoch 7/10
100/100 [==============================] - 452s 5s/step - loss: 0.2842 - acc: 0.8812 - val_loss: 

In [20]:
model.save('VGG16_TL.h5')

In [26]:
!conda list tensorflow 

# packages in environment at C:\Users\admin\Anaconda3:
#
# Name                    Version                   Build  Channel
tensorflow                1.15.0                   pypi_0    pypi
tensorflow-estimator      1.15.1                   pypi_0    pypi
tensorflow-hub            0.8.0                    pypi_0    pypi


In [25]:
!pip list tensorflow

Package                            Version            
---------------------------------- -------------------
absl-py                            0.9.0              
affine                             2.3.0              
aiohttp                            3.6.2              
alabaster                          0.7.12             
anaconda-client                    1.7.2              
anaconda-navigator                 1.9.12             
anaconda-project                   0.8.3              
APScheduler                        3.6.3              
argh                               0.26.2             
asn1crypto                         1.3.0              
astor                              0.8.1              
astroid                            2.3.3              
astropy                            4.0.1.post1        
async-timeout                      3.0.1              
atomicwrites                       1.3.0              
attrs                              19.3.0             
autopep8  

In [21]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

In [22]:
from keras.models import load_model
fm = load_model('VGG16_TL.h5')

TypeError: __init__() got an unexpected keyword argument 'ragged'

## 모델 훈련

이제 모델을 몇 번의 에포크로 훈련시키고 그 성능을 측정해 보자. 다음 코드로 모델에 새로 추가된 층을 훈련시키기 위한 fit_generator() 함수를 호출한다.

In [24]:
# predictions = model.predict(X_test/255.)

In [25]:
test_labels = list(y_test.squeeze())
predictions = list(predictions.argmax(axis=1))

In [ ]:
# label_dict = {0:'airplane',
#              1:'automobile',
#              2:'bird',
#              3:'cat',
#              4:'deer',
#              5:'dog',
#              6:'frog',
#              7:'horse',
#              8:'ship',
#              9:'truck'}

In [ ]:
# utils.plot_predictions(model=model,dataset=X_test/255.,
#                        dataset_labels=Y_test,
#                        label_dict=label_dict,
#                        batch_size=16,
#                        grid_height=4,
#                        grid_width=4)